In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import pickle
import os, re
import nltk
BASE_DIR = '../input/'
LABELED_TRAIN_DF = BASE_DIR + 'labeledTrainData.tsv'
UNLABELED_TRAIN_DF = BASE_DIR + 'unlabeledTrainData.tsv'
TEST_DF = BASE_DIR + 'testData.tsv'
print(os.listdir(BASE_DIR))

['word2vec_model_300dim_40minwords_10context', 'sampleSubmission.csv', 'labeled_train_clean_reviews.csv', 'word2vec_model_300dim_40minwords_10context_stemmed', 'labeledTrainData.tsv', 'word2vec_model_300dim_40minwords_10context_new', 'sentences_for_word2vec.csv', 'fasttext_300features_40minwords_10context.vec', 'labeled_train_clean_reviews_stemmed.csv', 'fasttext_300features_40minwords_10context.bin', 'test_submission.csv', 'testData.tsv', 'cc.en.300.bin', 'unlabeledTrainData.tsv', 'test_clean_reviews.csv', 'sentences_for_word2vec_stemmed.csv', 'sentences_for_fasttext.txt', 'test_clean_reviews_stemmed.csv']


In [2]:
labeled_train = pd.read_csv(LABELED_TRAIN_DF, header = 0, delimiter = '\t', quoting=3)
unlabeled_train = pd.read_csv(UNLABELED_TRAIN_DF, header = 0, delimiter = '\t', quoting=3)
test = pd.read_csv(TEST_DF, header = 0, delimiter = '\t', quoting=3)
print "Read %d labeled train reviews, %d unlabeled train reviews, " \
          "and %d test reviews" % (labeled_train["review"].size, unlabeled_train["review"].size, test["review"].size)

Read 25000 labeled train reviews, 50000 unlabeled train reviews, and 25000 test reviews


Data leakage

Check if test["sentiment"] is correct 

In [3]:
test["sentiment"] = test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)
y_test = test["sentiment"]

Credits: Kaggle tutorial

In [4]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

def review_to_clean_review(review, remove_stopwords=False, remove_numbers=True, stem_words=False):
    review_text = BeautifulSoup(review, "lxml").get_text()
    review_text = re.sub("[^a-zA-Z0-9]", " ", review_text)
    
    if remove_numbers:
        review_text = re.sub("[0-9]", " ", review_text)
    else:
        review_text = review_text.replace('10', ' ten ')
        review_text = review_text.replace('0', ' zero ')
        review_text = review_text.replace('1', ' one ')
        review_text = review_text.replace('2', ' two ')
        review_text = review_text.replace('3', ' three ')
        review_text = review_text.replace('4', ' four ')
        review_text = review_text.replace('5', ' five ')
        review_text = review_text.replace('6', ' six ')
        review_text = review_text.replace('7', ' seven ')
        review_text = review_text.replace('8', ' eight ')
        review_text = review_text.replace('9', ' nine ')
    
    review_text = review_text.lower()
    
    if remove_stopwords:
        words = review_text.split()
        stops = set(stopwords.words('english'))
        words = [w for w in words if not w in stops]
        review_text = " ".join(words)
    
    if stem_words:
        words = review_text.split()
        stemmer = SnowballStemmer('english')
        words = [stemmer.stem(word) for word in words]
        review_text = " ".join(words)
    
    return(review_text.strip())

def review_to_wordlist(review, remove_stopwords=False, remove_numbers=True, stem_words=False):

    words = review_to_clean_review(review, remove_stopwords, remove_numbers, stem_words).split()
    return words

def review_to_sentences(review, tokenizer, remove_stopwords=False, remove_numbers=True, stem_words=False):
    
    raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
    
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords, remove_numbers, stem_words))
    return sentences

In [5]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
example_review = "This is an example of a movie review. It has two sentences."
print(review_to_sentences(example_review, tokenizer, remove_numbers=False, stem_words=True))

[[u'this', u'is', u'an', u'exampl', u'of', u'a', u'movi', u'review'], [u'it', u'has', u'two', u'sentenc']]


In [6]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# Initialize an empty list of sentences
stemmed_sentences = []
sentences = []
# Parsing sentences from training set
counter = 0.
for review in labeled_train["review"]:
#     stemmed_sentences += review_to_sentences(review, tokenizer, remove_stopwords=False, remove_numbers=False, stem_words=True)
    sentences += review_to_sentences(review, tokenizer, remove_stopwords=False, remove_numbers=False, stem_words=False)
    if counter % 5000. == 0.:
        print "Labled train review %d of %d" % (counter, len(labeled_train["review"]))
    counter = counter + 1.

counter = 0.
# Parsing sentences from unlabeled set
for review in unlabeled_train["review"]:
#     stemmed_sentences += review_to_sentences(review, tokenizer, remove_stopwords=False, remove_numbers=False, stem_words=True)
    sentences += review_to_sentences(review, tokenizer, remove_stopwords=False, remove_numbers=False, stem_words=False)
    if counter % 5000. == 0.:
        print "Unlabled train review %d of %d" % (counter, len(unlabeled_train["review"]))
    counter = counter + 1.

Labled train review 0 of 25000


/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:272: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:335: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Labled train review 5000 of 25000
Labled train review 10000 of 25000
Labled train review 15000 of 25000
Labled train review 20000 of 25000
Unlabled train review 0 of 50000


/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:335: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:335: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Unlabled train review 5000 of 50000
Unlabled train review 10000 of 50000
Unlabled train review 15000 of 50000
Unlabled train review 20000 of 50000


...

/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:272: UserWarning: ".." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


Unlabled train review 35000 of 50000


/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:335: UserWarning: "http://www.youtube.com/watch?v=a0KSqelmgN8"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Unlabled train review 40000 of 50000
Unlabled train review 45000 of 50000


/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:335: UserWarning: "http://jake-weird.blogspot.com/2007/08/beneath.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [7]:
print(np.random.choice(sentences, 10))

[list([u'a', u'spin', u'off', u'of', u'donald', u'p', u'bellisario', u's', u'military', u'show', u'jag', u'ncis', u'is', u'a', u'favourite', u'of', u'mine', u'at', u'the', u'moment'])
 list([u'i', u'just', u'don', u't', u'know', u'how', u'i', u'could', u'expect', u'something', u'worth', u'a', u'look', u'from', u'a', u'film', u'with', u'such', u'plot', u'two', u'stupid', u'ignorant', u'kids', u'make', u'a', u'bet', u'that', u'each', u'of', u'them', u'will', u'do', u'something', u'certainly', u'extremely', u'idiotic', u'to', u'prove', u'to', u'each', u'other', u'wtf'])
 list([u'there', u'to', u'be', u'picked', u'off', u'at', u'a', u'moment', u's', u'notice'])
 list([u'talk', u'about', u'a', u'change', u'of', u'attitude', u'in', u'these', u'children', u's', u'adolescence'])
 list([u'she', u'is', u'hardly', u'in', u'the', u'film', u'at', u'all', u'which', u'is', u'the', u'movie', u's', u'only', u'saving', u'grace', u'and', u'the', u'ten', u'minutes', u'or', u'so', u'they', u'used', u'her',

In [8]:
joined_sentences = [" ".join(sentence).strip() for sentence in sentences]
output_sentences = pd.DataFrame(data={"sentences": joined_sentences})
output_sentences.to_csv(os.path.join('../', 'input', "sentences_for_word2vec_convert_10.csv"), index=False)

In [9]:
labeled_train_clean_reviews = []
counter = 0.
for review in labeled_train["review"]:
#     clean_review_stemmed = review_to_clean_review(review, remove_stopwords=False, remove_numbers=False, stem_words=True)
    clean_review = review_to_clean_review(review, remove_stopwords=False, remove_numbers=False, stem_words=False)
    labeled_train_clean_reviews.append(clean_review)
    if counter % 5000. == 0.:
        print "Train review %d of %d" % (counter, len(labeled_train["review"]))
    counter = counter + 1.

test_clean_reviews = []
counter = 0.
for review in test["review"]:
#     clean_review_stemmed = review_to_clean_review(review, remove_stopwords=False, remove_numbers=False, stem_words=True)
    clean_review = review_to_clean_review(review, remove_stopwords=False, remove_numbers=False, stem_words=False)
    test_clean_reviews.append(clean_review)
    if counter % 5000. == 0.:
        print "Test review %d of %d" % (counter, len(test["review"]))
    counter = counter + 1.

Train review 0 of 25000
Train review 5000 of 25000
Train review 10000 of 25000
Train review 15000 of 25000
Train review 20000 of 25000
Test review 0 of 25000
Test review 5000 of 25000
Test review 10000 of 25000
Test review 15000 of 25000
Test review 20000 of 25000


In [10]:
print(np.random.choice(labeled_train_clean_reviews, 10))
print(np.random.choice(test_clean_reviews, 10))

[u'i live in salt lake city and i m not a mormon  so why did i rent this movie  well because i live in utah and thought it d be nice to see locations i know in a film  i really knew going into it that i wasn t going to get the inside jokes so i wasn t surprised when i sat with the deer in the headlights stare  what i was surprised at was the ant non mormon actions that were placed in this film i know it s a mormon film  catered to the members of the lds church  but i found it offensive because of the typical stereotype of people that isn t of their faith  every non mormon  which wasn t many  drank  smoked and had an amazing selfishness attitude  why that really ticked me off about this film  they made the mormons so pure  yet the rest of the state of utah i guess is filled with punk psychos just because they don t follow the scriptures of the lds church i can understand having the plots revolve around all lds members  but you d think salt lake city was  ten  zero   mormon  which isn t 

In [11]:
output_labeled_train_clean_reviews = \
    pd.DataFrame(data={"id": labeled_train["id"], "sentiment": labeled_train["sentiment"], "review": labeled_train_clean_reviews})
output_test_clean_reviews = \
    pd.DataFrame(data={"id": test["id"], "review": test_clean_reviews})
# output_labeled_train_clean_reviews.to_csv(os.path.join('../', 'input', "labeled_train_clean_reviews_stemmed.csv"), index=False, quoting=3)
# output_test_clean_reviews.to_csv(os.path.join('../', 'input', "test_clean_reviews_stemmed.csv"), index=False, quoting=3)

output_labeled_train_clean_reviews.to_csv(os.path.join('../', 'input', "labeled_train_clean_reviews_convert_10.csv"), index=False, quoting=3)
output_test_clean_reviews.to_csv(os.path.join('../', 'input', "test_clean_reviews_convert_10.csv"), index=False, quoting=3)

Training the Word2Vec model

In [12]:
# Import the built-in logging module and configure it so that Word2Vec
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', \
                    level=logging.INFO)

from gensim.models import Word2Vec
EMBEDDING_DIM = 300  # Word vector dimensionality
MIN_WORD_COUNT = 40  # Minimum word count. Kaggle set to 40, to avoid attaching too much importance to individual movie titles.
NUM_THREADS = 4  # Number of threads to run in parallel
CONTEXT = 10  # Context window size
DOWNSAMPLING = 1e-3  # Downsample setting for frequent words
WORD2VEC_MODEL_FILE = BASE_DIR + \
    "word2vec_model_" + \
    str(EMBEDDING_DIM) + "dim_" + \
    str(MIN_WORD_COUNT) + "minwords_" + \
    str(CONTEXT) + "context_" +\
    "convert_10"

print "Training the Word2Vec model..."
model = Word2Vec(sentences, workers=NUM_THREADS, \
                 size=EMBEDDING_DIM, min_count=MIN_WORD_COUNT, \
                 window=CONTEXT, sample=DOWNSAMPLING, seed=1)
model.save(WORD2VEC_MODEL_FILE)


2019-01-10 05:40:30,962 : INFO : 'pattern' package found; tag filters are available for English
2019-01-10 05:40:30,970 : INFO : collecting all words and their counts
2019-01-10 05:40:30,972 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-01-10 05:40:31,052 : INFO : PROGRESS: at sentence #10000, processed 228974 words, keeping 17776 word types
2019-01-10 05:40:31,132 : INFO : PROGRESS: at sentence #20000, processed 457937 words, keeping 24948 word types


Training the Word2Vec model...


2019-01-10 05:40:31,214 : INFO : PROGRESS: at sentence #30000, processed 680112 words, keeping 30034 word types
2019-01-10 05:40:31,298 : INFO : PROGRESS: at sentence #40000, processed 909365 words, keeping 34348 word types
2019-01-10 05:40:31,377 : INFO : PROGRESS: at sentence #50000, processed 1131511 words, keeping 37761 word types
2019-01-10 05:40:31,456 : INFO : PROGRESS: at sentence #60000, processed 1355867 words, keeping 40723 word types
2019-01-10 05:40:31,535 : INFO : PROGRESS: at sentence #70000, processed 1581925 words, keeping 43333 word types
2019-01-10 05:40:31,614 : INFO : PROGRESS: at sentence #80000, processed 1804032 words, keeping 45714 word types
2019-01-10 05:40:31,691 : INFO : PROGRESS: at sentence #90000, processed 2031179 words, keeping 48135 word types
2019-01-10 05:40:31,765 : INFO : PROGRESS: at sentence #100000, processed 2255867 words, keeping 50207 word types
2019-01-10 05:40:31,842 : INFO : PROGRESS: at sentence #110000, processed 2478609 words, keeping 

2019-01-10 05:40:36,610 : INFO : PROGRESS: at sentence #750000, processed 16987629 words, keeping 120295 word types
2019-01-10 05:40:36,678 : INFO : PROGRESS: at sentence #760000, processed 17209786 words, keeping 120930 word types
2019-01-10 05:40:36,754 : INFO : PROGRESS: at sentence #770000, processed 17439959 words, keeping 121703 word types
2019-01-10 05:40:36,833 : INFO : PROGRESS: at sentence #780000, processed 17673154 words, keeping 122402 word types
2019-01-10 05:40:36,905 : INFO : PROGRESS: at sentence #790000, processed 17903065 words, keeping 123066 word types
2019-01-10 05:40:36,952 : INFO : collected 123504 word types from a corpus of 18027659 raw words and 795538 sentences
2019-01-10 05:40:36,954 : INFO : Loading a fresh vocabulary
2019-01-10 05:40:37,067 : INFO : effective_min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2019-01-10 05:40:37,068 : INFO : effective_min_count=40 leaves 17468517 word corpus (96% of original 18027659, drops 559

2019-01-10 05:41:29,793 : INFO : EPOCH 3 - PROGRESS: at 41.77% examples, 595344 words/s, in_qsize 8, out_qsize 0
2019-01-10 05:41:30,796 : INFO : EPOCH 3 - PROGRESS: at 46.41% examples, 595960 words/s, in_qsize 8, out_qsize 0
2019-01-10 05:41:31,805 : INFO : EPOCH 3 - PROGRESS: at 51.02% examples, 596198 words/s, in_qsize 7, out_qsize 0
2019-01-10 05:41:32,805 : INFO : EPOCH 3 - PROGRESS: at 55.61% examples, 596161 words/s, in_qsize 8, out_qsize 0
2019-01-10 05:41:33,813 : INFO : EPOCH 3 - PROGRESS: at 60.07% examples, 595176 words/s, in_qsize 8, out_qsize 0
2019-01-10 05:41:34,822 : INFO : EPOCH 3 - PROGRESS: at 64.68% examples, 594919 words/s, in_qsize 8, out_qsize 0
2019-01-10 05:41:35,822 : INFO : EPOCH 3 - PROGRESS: at 69.25% examples, 594996 words/s, in_qsize 8, out_qsize 0
2019-01-10 05:41:36,823 : INFO : EPOCH 3 - PROGRESS: at 73.84% examples, 595063 words/s, in_qsize 8, out_qsize 0
2019-01-10 05:41:37,828 : INFO : EPOCH 3 - PROGRESS: at 78.60% examples, 596199 words/s, in_qsiz

Generate the txt file for FastText

In [13]:
sentences = pd.read_csv("../input/sentences_for_word2vec_convert_10.csv", header = 0)
sentences["sentences"] = sentences["sentences"].astype(str)
sentences = sentences["sentences"].tolist()
print(np.random.choice(sentences, 10))

['that s supposed to be entertainment'
 'from the start the best actors are alec newman and joe absolom this is what british films are about good actors low money made into something decent'
 'of course he redeems himself but only in the final episode and for no apparent reason'
 'there is one or two decent performances including the guys who played jacob and fenner but it s like the director had no clue on how to work or use his thespians'
 'dustin hoffman s performance as comic lenny bruce is brilliant and i agree with those who say he is one of the greatest actors in cinema history'
 'i found it it sweet funny and adorable'
 'the men can slaver over lollo if they like or her lollos she gave her name to a slang terms for breasts in french but the ladies have an even tastier morsel in the divine gerard philipe who is not only beautiful but can act'
 'francesca annis is one of the most striking five zero ish women imaginable her acting rivals her beauty'
 'i had enjoyed all three films

In [14]:
output_sentences = open('../input/sentences_for_fasttext_convert_10.txt', 'w')
for sentence in sentences:
    output_sentences.write(sentence + '\n')
output_sentences.close()